## 0.关联性简单理解--牛奶面包

In [40]:
#使用工具包：
from efficient_apriori import apriori
#或者：
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

from efficient_apriori import apriori
# 设置数据集
transactions = [('牛奶','面包','尿布'),
		('可乐','面包', '尿布', '啤酒'),
		('牛奶','尿布', '啤酒', '鸡蛋'),
		('面包', '牛奶', '尿布', '啤酒'),
		('面包', '牛奶', '尿布', '可乐')]
# 挖掘频繁项集和频繁规则
itemsets, rules = apriori(transactions, min_support=0.5,  min_confidence=1)
print("频繁项集：", itemsets)
print("关联规则：", rules)

频繁项集： {1: {('牛奶',): 4, ('尿布',): 5, ('面包',): 4, ('啤酒',): 3}, 2: {('尿布', '牛奶'): 4, ('尿布', '面包'): 4, ('牛奶', '面包'): 3, ('啤酒', '尿布'): 3}, 3: {('尿布', '牛奶', '面包'): 3}}
关联规则： [{牛奶} -> {尿布}, {面包} -> {尿布}, {啤酒} -> {尿布}, {牛奶, 面包} -> {尿布}]


## 1.购物篮相关item分析

- **efficient_apriori工具箱调用apriori方法**

In [19]:
from efficient_apriori import apriori
# #或者：
# from mlxtend.frequent_patterns import apriori
# from mlxtend.frequent_patterns import association_rules
import pandas as pd

# 数据加载
data = pd.read_csv(r'D:\i学习\Data_Engine_with_Python-master\L4\BreadBasket\BreadBasket_DMS.csv')
# 统一小写
data['Item'] = data['Item'].str.lower()
data[data.Item == 'none'].index   
# 去掉none项
data = data.drop(data[data.Item == 'none'].index)   #删除行：df.drop([行的索引]，inplace=TRUE/FALSE);删除列：df.drop(['col1',col2],axis = 1,inplace = )

# 得到一维数组orders_series，并且将Transaction作为index(因为Transaction不是特征值）, value为Item取值
orders_series = data.set_index('Transaction')['Item']   # data.set_index('Transaction')之后得到一个索引变了的DataFrame，然后再选择item这一列
orders_series
#将数据集进行格式转换
transactions = []
temp_index = 0
for i, v in orders_series.items():
	if i != temp_index:
		temp_set = set()      #set：无序，不重复，可修改（存储Key)；dict（存储key和value)无序可重复可修改;list：有序，可重复，可修改(存储同结构数据）；tuple：有序，可重复，不可修改（存储异构数据）
		temp_index = i
		temp_set.add(v)
		transactions.append(temp_set)
	else:
		temp_set.add(v)
#print(transactions)
#print(type(transactions))
itemsets, rules = apriori(transactions, min_support=0.02,  min_confidence=0.5)   #导入模块仅导入要使用的就行，不能导入多个不同模块从中待用同一个函数，因为函数不不一样可能报错
#通过调整min_support，min_confidence可以得到不同的频繁项集和关联规则
print('频繁项集:',itemsets)
print('关联规则：',rules)

频繁项集: {1: {('scandinavian',): 275, ('cookies',): 515, ('hot chocolate',): 552, ('muffin',): 364, ('bread',): 3096, ('coffee',): 4528, ('pastry',): 815, ('medialuna',): 585, ('tea',): 1350, ('farm house',): 371, ('juice',): 365, ('soup',): 326, ('cake',): 983, ('sandwich',): 680, ('alfajores',): 344, ('brownie',): 379, ('truffles',): 192, ('toast',): 318, ('scone',): 327}, 2: {('bread', 'coffee'): 852, ('bread', 'pastry'): 276, ('coffee', 'pastry'): 450, ('coffee', 'medialuna'): 333, ('coffee', 'tea'): 472, ('bread', 'tea'): 266, ('coffee', 'juice'): 195, ('coffee', 'hot chocolate'): 280, ('coffee', 'cookies'): 267, ('cake', 'coffee'): 518, ('cake', 'tea'): 225, ('bread', 'cake'): 221, ('coffee', 'sandwich'): 362, ('coffee', 'toast'): 224}}
关联规则： [{pastry} -> {coffee}, {medialuna} -> {coffee}, {juice} -> {coffee}, {hot chocolate} -> {coffee}, {cookies} -> {coffee}, {cake} -> {coffee}, {sandwich} -> {coffee}, {toast} -> {coffee}]


- **mlxtend工具箱调用apriori方法**

In [38]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# 数据加载
data = pd.read_csv(r'D:\i学习\Data_Engine_with_Python-master\L4\BreadBasket\BreadBasket_DMS.csv')
#数据清洗
# 统一小写
data['Item'] = data['Item'].str.lower()
data[data.Item == 'none'].index   
# 去掉none项
data = data.drop(data[data.Item == 'none'].index)   #删除行：df.drop([行的索引]，inplace=TRUE/FALSE);删除列：df.drop(['col1',col2],axis = 1,inplace = )


#grouped = data.groupby(['Transaction','Item'])['Item'].count()
#grouped
#转化成适合One hot编码的样式，通过行列透视实现
hot_encoded_df=data.groupby(['Transaction','Item'])['Item'].count().unstack().reset_index().fillna(0).set_index('Transaction')   #unstack()将行中数据透视到列;stack()将列中数据透视到行；也可用pivot实现

#print(hot_encoded_df)
#one hot编码函数转换
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
#函数encode用在每一个数据上，实现转换    
hot_encoded_df = hot_encoded_df.applymap(encode_units)
# print(hot_encoded_df)
#使用mlxtend工具箱的apriori关联方法
frequent_itemsets = apriori(hot_encoded_df, min_support=0.02, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.5)

print('频繁项集:',frequent_itemsets)
print('关联规则：',rules)

频繁项集:      support                 itemsets
0   0.036348              (alfajores)
1   0.327134                  (bread)
2   0.040046                (brownie)
3   0.103867                   (cake)
4   0.478445                 (coffee)
5   0.054417                (cookies)
6   0.039201             (farm house)
7   0.058326          (hot chocolate)
8   0.038567                  (juice)
9   0.061813              (medialuna)
10  0.038462                 (muffin)
11  0.086116                 (pastry)
12  0.071851               (sandwich)
13  0.029057           (scandinavian)
14  0.034552                  (scone)
15  0.034446                   (soup)
16  0.142646                    (tea)
17  0.033601                  (toast)
18  0.020287               (truffles)
19  0.023352            (bread, cake)
20  0.090025          (bread, coffee)
21  0.029163          (bread, pastry)
22  0.028107             (bread, tea)
23  0.054734           (coffee, cake)
24  0.023774              (tea, cake)
25  0.

**其他-过程理解**

In [31]:
hot_encoded_df=data.groupby(['Transaction','Item'])['Item'].count().unstack().reset_index()
print(hot_encoded_df)

Item  Transaction  adjustment  afternoon with the baker  alfajores  \
0               2         NaN                       NaN        NaN   
1               3         NaN                       NaN        NaN   
2               4         NaN                       NaN        NaN   
3               5         NaN                       NaN        NaN   
4               6         NaN                       NaN        NaN   
...           ...         ...                       ...        ...   
9459         9680         NaN                       NaN        NaN   
9460         9681         NaN                       NaN        NaN   
9461         9682         NaN                       NaN        NaN   
9462         9683         NaN                       NaN        NaN   
9463         9684         NaN                       NaN        NaN   

Item  argentina night  art tray  bacon  baguette  bakewell  bare popcorn  ...  \
0                 NaN       NaN    NaN       NaN       NaN           NaN  ... 

In [30]:
data['Item'].value_counts()

coffee            5471
bread             3324
tea               1435
cake              1025
pastry             856
                  ... 
chicken sand         1
raw bars             1
adjustment           1
gift voucher         1
olum & polenta       1
Name: Item, Length: 94, dtype: int64

In [39]:
?apriori

## 2.电影分类中的关联分析

In [50]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# 数据加载
data = pd.read_csv(r'D:\i学习\Data_Engine_with_Python-master\L4\MovieLens\movies.csv')
#数据转化：将genres进行one-hot编码（离散特征有多少取值，就用多少维来表示这个特征）
movies_hot_encoded = data.drop('genres',axis = 1).join(data.genres.str.get_dummies('|'))

# 将movieId, title设置为index（不是特征值的列要设置为索引，某则会误认为特征）
movies_hot_encoded.set_index(['movieId','title'],inplace=True)
movies_hot_encoded
# 挖掘频繁项集，最小支持度为0.02
itemsets = apriori(movies_hot_encoded,use_colnames=True, min_support=0.02)
itemsets_sort = itemsets.sort_values(by = 'support',ascending = False)
# 根据频繁项集计算关联规则，设置最小提升度为2
rules =  association_rules(itemsets, metric='lift', min_threshold=2)
rules_sort = rules.sort_values(by = 'lift',ascending = False)

print('频繁项集:',itemsets_sort)
print('关联规则：',rules_sort)


频繁项集:      support                  itemsets
7   0.489185                   (Drama)
4   0.306987                  (Comedy)
14  0.153164                (Thriller)
12  0.151294                 (Romance)
0   0.129042                  (Action)
5   0.107743                   (Crime)
9   0.095718                  (Horror)
31  0.094325          (Romance, Drama)
26  0.093335           (Comedy, Drama)
6   0.090586             (Documentary)
1   0.085380               (Adventure)
27  0.069470         (Comedy, Romance)
32  0.068480         (Thriller, Drama)
13  0.063898                  (Sci-Fi)
28  0.062761            (Crime, Drama)
11  0.055503                 (Mystery)
8   0.051763                 (Fantasy)
29  0.045165         (Crime, Thriller)
20  0.044101           (Action, Drama)
15  0.043772                     (War)
3   0.041755                (Children)
22  0.040655        (Action, Thriller)
34  0.039336        (Horror, Thriller)
10  0.037979                 (Musical)
2   0.037649       

## 3.电影演员中的关联关系

In [57]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# 数据加载
data = pd.read_csv(r'D:\i学习\Data_Engine_with_Python-master\L4\MovieActors\movie_actors.csv')
#数据转化：将genres进行one-hot编码（离散特征有多少取值，就用多少维来表示这个特征）
movies_hot_encoded = data.drop('actors',axis = 1).join(data.actors.str.get_dummies('/'))

# 将movieId, title设置为index
movies_hot_encoded.set_index(['title'],inplace=True)

# 挖掘频繁项集，最小支持度为0.05
itemsets = apriori(movies_hot_encoded,use_colnames=True, min_support=0.05)
# 按照支持度从大到小进行排序
itemsets = itemsets.sort_values(by="support" , ascending=False) 

pd.options.display.max_columns=100
# 根据频繁项集计算关联规则，设置最小提升度为2
rules =  association_rules(itemsets, metric='lift', min_threshold=2)
# 按照提升度从大到小进行排序
rules = rules.sort_values(by="lift" , ascending=False) 
#rules.to_csv('./rules.csv')

print('频繁项集:',itemsets)
print('关联规则：',rules)

频繁项集:      support      itemsets
2   0.768421          (徐峥)
7   0.157895          (黄渤)
11  0.094737      (黄渤, 徐峥)
1   0.073684          (宁浩)
0   0.063158         (于和伟)
6   0.063158          (陶虹)
8   0.063158      (宁浩, 徐峥)
9   0.063158      (黄渤, 宁浩)
13  0.063158  (黄渤, 宁浩, 徐峥)
3   0.052632         (王宝强)
4   0.052632          (王迅)
5   0.052632         (陈凯歌)
10  0.052632      (陶虹, 徐峥)
12  0.052632      (黄渤, 王迅)
关联规则：   antecedents consequents  antecedent support  consequent support   support  \
2    (黄渤, 徐峥)        (宁浩)            0.094737            0.073684  0.063158   
5        (宁浩)    (黄渤, 徐峥)            0.073684            0.094737  0.063158   
3    (宁浩, 徐峥)        (黄渤)            0.063158            0.157895  0.063158   
7        (王迅)        (黄渤)            0.052632            0.157895  0.052632   
4        (黄渤)    (宁浩, 徐峥)            0.157895            0.063158  0.063158   
6        (黄渤)        (王迅)            0.157895            0.052632  0.052632   
0        (黄渤)        (宁浩)     

## 4.汽车碰撞安全测试中的各指标之间的关系

In [76]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# 数据加载
data = pd.read_csv(r'D:\i学习\Data_Engine_with_Python-master\L4\ncap\ncap.csv',encoding = 'gbk')   #中文为避免乱码及报错，需设置编码格式encoding = 'gbk'
data.set_index('车型',inplace = True)
data
#数据转化：将value进行0/1转换
f = lambda x : 1 if int(x) >= 12 else 0 
cransh_test = data.applymap(f)

# 挖掘频繁项集，最小支持度为0.05
itemsets = apriori(cransh_test,use_colnames=True, min_support=0.05)
# 按照支持度从大到小进行排序
itemsets = itemsets.sort_values(by="support" , ascending=False) 

# pd.options.display.max_columns=100
# 根据频繁项集计算关联规则，设置最小提升度为2
rules =  association_rules(itemsets, metric='lift', min_threshold=0.5)
# 按照置信度从大到小进行排序（注意置信度与提升度含义不同，不同业务场景需要看的指标也不同）
rules = rules.sort_values(by="confidence" , ascending=False) 
#rules.to_csv('./rules.csv')

print('频繁项集:',itemsets)
print('关联规则：',rules)

频繁项集:     support                   itemsets
1  0.866667                  (正面40%重叠)
2  0.666667                     (侧面碰撞)
0  0.600000                 (正面100%重叠)
3  0.600000        (正面100%重叠, 正面40%重叠)
5  0.600000            (正面40%重叠, 侧面碰撞)
4  0.466667           (正面100%重叠, 侧面碰撞)
6  0.466667  (正面100%重叠, 正面40%重叠, 侧面碰撞)
关联规则：             antecedents          consequents  antecedent support  \
0            (正面100%重叠)            (正面40%重叠)            0.600000   
7      (正面100%重叠, 侧面碰撞)            (正面40%重叠)            0.466667   
3                (侧面碰撞)            (正面40%重叠)            0.666667   
4            (正面100%重叠)               (侧面碰撞)            0.600000   
6   (正面100%重叠, 正面40%重叠)               (侧面碰撞)            0.600000   
8       (正面40%重叠, 侧面碰撞)           (正面100%重叠)            0.600000   
9            (正面100%重叠)      (正面40%重叠, 侧面碰撞)            0.600000   
5                (侧面碰撞)           (正面100%重叠)            0.666667   
11               (侧面碰撞)  (正面100%重叠, 正面40%重叠)            0.666667

In [88]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# 数据加载
dataset = pd.read_csv(r'D:\i学习\Data_Engine_with_Python-master\L4\MarketBasket\Market_Basket_Optimisation.csv',header=None,encoding = 'gbk')   #中文为避免乱码及报错，需设置编码格式encoding = 'gbk'
# data.set_index('车型',inplace = True)
# newname = '/'.join(dataset.iloc[1,:].values)
# print(newname)
dataset['new_col'] = dataset.apply(lambda x: '-'.join(x.dropna().astype(str).values), axis=1)
df = dataset[['new_col']]
result = df.drop('new_col', 1).join(dataset.new_col.str.get_dummies('-'))
# result

# 挖掘频繁项集，最小支持度为0.05
itemsets = apriori(result,use_colnames=True, min_support=0.05)
# 按照支持度从大到小进行排序
itemsets = itemsets.sort_values(by="support" , ascending=False) 

# pd.options.display.max_columns=100
# 根据频繁项集计算关联规则，设置最小提升度为2
rules =  association_rules(itemsets, metric='lift', min_threshold=0.5)
# 按照置信度从大到小进行排序（注意置信度与提升度含义不同，不同业务场景需要看的指标也不同）
rules = rules.sort_values(by="confidence" , ascending=False) 
#rules.to_csv('./rules.csv')

print('频繁项集:',itemsets)
print('关联规则：',rules)


频繁项集:      support                    itemsets
16  0.238368             (mineral water)
6   0.179709                      (eggs)
21  0.174110                 (spaghetti)
8   0.170911              (french fries)
3   0.163845                 (chocolate)
12  0.132116                 (green tea)
15  0.129583                      (milk)
13  0.098254               (ground beef)
10  0.095321         (frozen vegetables)
18  0.095054                  (pancakes)
0   0.087188                   (burgers)
1   0.081056                      (cake)
4   0.080389                   (cookies)
7   0.079323                  (escalope)
14  0.076523            (low fat yogurt)
19  0.071457                    (shrimp)
22  0.068391                  (tomatoes)
17  0.065858                 (olive oil)
9   0.063325           (frozen smoothie)
23  0.062525                    (turkey)
2   0.059992                   (chicken)
27  0.059725  (spaghetti, mineral water)
24  0.058526          (whole wheat rice)
25  0.0526

## 相关性

In [2]:
# 构造一元二次方程，y=2x*x+1 非线性关系
import pandas as pd
def compute(x):
    return 2*x*x+1
x=[i for i in range(100)]
y=[compute(i) for i in x]
data = pd.DataFrame({'x':x,'y':y})
# 查看pearson系数
print(data.corr())
# 查看method等于其他系数的方法
print(data.corr(method='spearman'))
print(data.corr(method='kendall'))

          x         y
x  1.000000  0.967644
y  0.967644  1.000000
     x    y
x  1.0  1.0
y  1.0  1.0
     x    y
x  1.0  1.0
y  1.0  1.0


In [3]:
?data.corr

## 回归分析

### 线性回归

### 一元线性回归

In [29]:

import random
import numpy as np
import pandas as pd
from sklearn import linear_model
reg = linear_model.LinearRegression()

#生成x和y（x必须为reshape到-1至1之间，或者必须为多维）
X = np.array([5, 15, 25, 35, 45, 55]).reshape((-1, 1))
y = np.array([5, 20, 14, 32, 22, 38])

#进行线性回归  
print('拟合参数：',reg.fit(X,y))
print('评分结果：',reg.score(X,y))
print('回归系数：',reg.coef_)
print('截距：',reg.intercept_)
# fit(X,y)，训练，拟合参数
# predict(X) ，预测
# coef_ ，存放回归系数
# intercept_，存放截距
# score(X,y)， 得到评分结果，R方（确定系数）


拟合参数： LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
评分结果： 0.715875613747954
回归系数： [0.54]
截距： 5.633333333333329


In [3]:
?LinearRegression

Object `LinearRegression` not found.


In [10]:
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
print(X.shape)

(4, 2)


In [21]:
# 回归分析
import random
from sklearn import linear_model
reg = linear_model.LinearRegression()

def generate(x):
	y = 2*x+10+random.random()
	return y

train_x = []
train_y = []
print(train_x,train_y)
for x in range(1000):
	train_x.append([x])
	y = generate(x)
	train_y.append([y])

print(train_x)
print(train_y)
# reg.fit (train_x, train_y)
# # coef_ 保存线性模型的系数w
# print(reg.coef_)
# print(reg.intercept_)

[] []
[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32], [33], [34], [35], [36], [37], [38], [39], [40], [41], [42], [43], [44], [45], [46], [47], [48], [49], [50], [51], [52], [53], [54], [55], [56], [57], [58], [59], [60], [61], [62], [63], [64], [65], [66], [67], [68], [69], [70], [71], [72], [73], [74], [75], [76], [77], [78], [79], [80], [81], [82], [83], [84], [85], [86], [87], [88], [89], [90], [91], [92], [93], [94], [95], [96], [97], [98], [99], [100], [101], [102], [103], [104], [105], [106], [107], [108], [109], [110], [111], [112], [113], [114], [115], [116], [117], [118], [119], [120], [121], [122], [123], [124], [125], [126], [127], [128], [129], [130], [131], [132], [133], [134], [135], [136], [137], [138], [139], [140], [141], [142], [143], [144], [145], [146], [147], [148], [149], [150], [151], [152], [153], [154], [155], [156], [157

### 多元线性回归

In [32]:
import random
import numpy as np
import pandas as pd
from sklearn import linear_model
reg = linear_model.LinearRegression()

#生成x和y(回归模型：自变量x可谓为多维，但是y必须为一维)
X = [[0, 1,3], [5, 1,4], [15, 2,3], [25, 5,6], [35, 11,13], [45, 15,18], [55, 34,36], [60, 35,37]]
y = [4, 5, 20, 14, 32, 22, 38, 43]

#进行线性回归  
print('拟合参数：',reg.fit(X,y))
print('评分结果：',reg.score(X,y))
print('回归系数：',reg.coef_)
print('截距：',reg.intercept_)


拟合参数： LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
评分结果： 0.8943013575442007
回归系数： [ 0.42435877  3.82142922 -3.5104962 ]
截距： 12.498100214797589


### 多项式回归

In [11]:
import random
import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures  #PolynomialFeatures 用于生成多项式和交互特征。
poly = PolynomialFeatures(2)

x = np.array([5, 15, 25, 35, 45, 55]).reshape((-1, 1))
y = np.array([15, 11, 2, 8, 25, 32])

poly.fit_transform(x)

array([[1.000e+00, 5.000e+00, 2.500e+01],
       [1.000e+00, 1.500e+01, 2.250e+02],
       [1.000e+00, 2.500e+01, 6.250e+02],
       [1.000e+00, 3.500e+01, 1.225e+03],
       [1.000e+00, 4.500e+01, 2.025e+03],
       [1.000e+00, 5.500e+01, 3.025e+03]])

In [9]:
?preprocessing.PolynomialFeatures

## 糖尿病回归分析

In [19]:
from sklearn import datasets
from sklearn import linear_model
from sklearn import metrics
# from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split  
# 加载数据
diabetes = datasets.load_diabetes()
data = diabetes.data
# 训练集 70%，测试集30%
train_x, test_x, train_y, test_y = train_test_split(diabetes.data, diabetes.target, test_size=0.3, random_state=14)
print(len(train_x))

#回归训练及预测
clf = linear_model.LinearRegression()
clf.fit(train_x, train_y)
print(clf.coef_)
pred_y = clf.predict(test_x)
print(mean_squared_error(test_y, pred_y)) #计算均方误差回归损失

309
[  32.03000032 -228.38626681  492.80665731  313.61844116 -991.31389923
  551.99413533  190.16297006  278.51146815  781.03825662   72.08348977]
3180.3670319563716


In [16]:
?metrics

In [25]:

from efficient_apriori import apriori
# #或者：
# from mlxtend.frequent_patterns import apriori
# from mlxtend.frequent_patterns import association_rules
import pandas as pd

# 数据加载
df1 = pd.read_csv(r'D:\i学习\Data_Engine_with_Python-master\L4\tianchi_usedCar\used_car_testB_20200421\used_car_testB_20200421.csv')
# 数据格式转换-单列拆分成多列
df2 = df1.iloc[:,0] .str.split(' ',expand = True) #选取第一列（可用列名选取/也可用索引选取）得series,再利用split进行拆分，expand=True将参数拆成多列
df2.columns = df1.columns.values[0].split()  #读取列名--选取第一个列名--拆分--重命名
df2

,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,...,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
0,200000,133777,20000501,67.0,0,1.0,0.0,0.0,101,15.0,...,0.23651953807463394,0.00024077965516603838,0.10531899027928028,0.04623333858358547,0.09452231271103513,3.619512125855613,-0.2806066537539741,-2.019761143295854,0.9788277260800712,0.8033215020878566
1,200001,61206,19950211,19.0,6,2.0,0.0,0.0,73,6.0,...,0.26151841976421497,0.0,0.12032345361861815,0.0467842378305852,0.035385262671275085,2.9973763596922285,-1.4067050523440334,-1.0208835817916766,-1.3499898633435856,-0.20054163936348302
2,200002,67829,20090606,5.0,5,4.0,0.0,0.0,120,5.0,...,0.26169061811955624,0.09083648656092408,0.0,0.07965532570737711,0.0735862207476284,-3.951083771010004,-0.4334673285213749,0.9189638428560336,1.6346039890078308,1.027172758680927
3,200003,8892,20020601,22.0,9,1.0,0.0,0.0,58,15.0,...,0.2360495075063573,0.10177689524069478,0.09894989511106032,0.026829627502826414,0.09661365556957097,-2.8467877718832733,2.8002670817288,-2.5246103235495783,1.0768192298469703,0.4616102367935517
4,200004,76998,20030301,46.0,6,0.0,,0.0,116,15.0,...,0.2569995358233025,0.0,0.06673175734217886,0.05777117201467578,0.06885246978026283,2.839010006118193,-1.6598006754576482,-0.9241417494176124,0.19942261240833112,0.4510139980592859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,249995,111443,20041005,4.0,4,0.0,,1.0,150,15.0,...,0.2636682962953293,0.00029201863808368887,0.14180363262285192,0.07639328122943874,0.03927205316101391,2.072900927766656,-2.531868588058039,1.7169780955424252,-1.0634370434847673,0.3265874651785571
49996,249996,152834,20130409,65.0,1,0.0,0.0,0.0,179,4.0,...,0.2553099445739612,0.000991151617020502,0.15586776360827434,0.10842482126822993,0.06784061274017303,1.3585039692608445,-3.290295216928372,4.269808939592735,0.1405236437690217,0.5562210331848007
49997,249997,132531,20041211,4.0,4,0.0,0.0,1.0,147,12.5,...,0.2629332648742647,0.0003177907734646262,0.1418715927583109,0.07196767659848803,0.042965543843221626,2.165658249768226,-2.4178854984041585,1.3706116252547165,-1.073132883304749,0.2706021302994931
49998,249998,143405,20020702,40.0,1,4.0,0.0,1.0,176,15.0,...,0.28210627445213776,2.3076355981708684e-05,0.06748298195221036,0.06752581288470348,0.009005552682365877,2.03011415062505,-2.939243980189933,0.5690778271208292,-1.7182451470319045,0.31637930050823204


In [13]:
c1 = df1.columns.values[0]
c2 = c1.split()
print(c2)

df1

['SaleID', 'name', 'regDate', 'model', 'brand', 'bodyType', 'fuelType', 'gearbox', 'power', 'kilometer', 'notRepairedDamage', 'regionCode', 'seller', 'offerType', 'creatDate', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13', 'v_14']


In [ ]:
clo_name = ['SaleID', 'name', 'regDate', 'model', 'brand', 'bodyType', 'fuelType', 
            'gearbox', 'power', 'kilometer', 'notRepairedDamage', 'regionCode', 'seller',
            'offerType', 'creatDate', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6', '
            v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13', 'v_14']